# Statystyczne Reguły Decyzyjne
## Projekt: analiza zbioru Adult
Laura Hoang 140381

### 1. Wprowadzanie i opis problemu

W ramach niniejszego projektu rozważany jest problem klasyfikacji binarnej, polegający na przewidywaniu poziomu dochodu osoby fizycznej na podstawie danych pochodzących ze spisu ludności. Do realizacji zadania wykorzystano klasyczny zbiór danych **Adult (Census Income)**, który stanowi standardowy punkt odniesienia w badaniach nad metodami klasyfikacyjnymi oraz regułami decyzyjnymi.

Rozpatrywany problem decyzyjny polega na **przewidywaniu, czy roczny dochód danej osoby przekracza 50 000 dolarów amerykańskich**, na podstawie zestawu jej cech demograficznych i ekonomicznych, pochodzących z danych spisowych. Formalnie jest to zadanie **klasyfikacji binarnej**, w którym każdej obserwacji przypisywana jest jedna z dwóch możliwych klas decyzyjnych:

* dochód **nieprzekraczający 50 000 USD rocznie** (`<=50K`),
* dochód **przekraczający 50 000 USD rocznie** (`>50K`).

Zmienna decyzyjna `income` stanowi etykietę klasy, natomiast pozostałe zmienne opisowe tworzą przestrzeń cech, na podstawie której konstruowany jest model klasyfikacyjny. Obejmują one m.in. wiek, poziom wykształcenia, formę zatrudnienia, wykonywany zawód, stan cywilny, liczbę przepracowanych godzin tygodniowo, a także cechy demograficzne takie jak płeć, rasa czy kraj pochodzenia.

Celem analizy jest wyznaczenie takiej reguły decyzyjnej, która na podstawie obserwowanego wektora cech maksymalizuje prawdopodobieństwo poprawnej klasyfikacji dochodu danej osoby. Problem ten ma istotne znaczenie praktyczne, ponieważ pozwala analizować zależności pomiędzy cechami demograficznymi a poziomem dochodów, a jednocześnie stanowi dobre studium przypadku do porównywania różnych metod klasyfikacji statystycznej.

Z punktu widzenia teorii statystycznych reguł decyzyjnych, zadanie to może być rozpatrywane jako problem minimalizacji ryzyka klasyfikacyjnego przy zadanej funkcji straty, co umożliwia formalną ocenę jakości zastosowanych modeli oraz porównanie ich skuteczności.

In [ ]:
# pobranie potrzebnych bibliotek
import pandas as pd
from ucimlrepo import fetch_ucirepo  # biblioteka z danymi

adult = fetch_ucirepo(id=2)  # zbiór danych

### 2. Opis zbioru danych

W projekcie wykorzystano zbiór danych **Adult** (znany również jako **Census Income Dataset**), pochodzący z repozytorium **UCI Machine Learning Repository**. Zbiór ten został opracowany na podstawie danych spisowych z **1994 roku**, a jego ekstrakcję przeprowadzili **Barry Becker** oraz **Ronny Kohavi**. Dane zostały opublikowane w **1996 roku**, a ich najnowsza aktualizacja miała miejsce **24 września 2024 roku**. Zbiór posiada identyfikator DOI: *10.24432/C5XW20*.

Celem zbioru danych jest rozwiązanie zadania **klasyfikacji binarnej**, polegającego na przewidywaniu, czy **roczny dochód danej osoby przekracza 50 000 USD**. Zmienna decyzyjna (`income`) przyjmuje dwie wartości: `>50K` oraz `<=50K`.

Zbiór danych należy do obszaru **nauk społecznych** i ma charakter **wielowymiarowy (multivariate)**. Składa się z **48 842 obserwacji** oraz **14 zmiennych objaśniających**, opisujących cechy demograficzne i ekonomiczne badanych osób. Zmienne mają charakter **liczbowy (integer)** oraz **kategoryczny** i obejmują informacje dotyczące m.in. wieku, poziomu wykształcenia, sytuacji zawodowej, stanu cywilnego, rasy, płci oraz liczby przepracowanych godzin w tygodniu.

Podczas ekstrakcji danych zastosowano zestaw warunków filtrujących w celu uzyskania względnie jednorodnego i „czystego” zbioru obserwacji. Do zbioru włączono jedynie rekordy spełniające następujące kryteria:

* wiek powyżej 16 lat,
* dochód brutto powyżej 100 USD,
* waga populacyjna (`fnlwgt`) większa od 1,
* liczba godzin pracy w tygodniu większa od 0.

W zbiorze danych występują **braki danych**, oznaczone symbolem `NaN`, głównie w zmiennych kategorycznych takich jak `workclass`, `occupation` oraz `native-country`.

| Zmienna            | Rola         | Typ danych   | Demografia           | Opis wartości                                                    | Braki danych |
| ------------------ | ------------ | ------------ | -------------------- | ---------------------------------------------------------------- | ------------ |
| **age**            | Cecha        | Liczbowa     | Wiek                 | *(brak)*                                                         | nie          |
| **workclass**      | Cecha        | Kategoryczna | Zarobki              | Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov… | tak          |
| **fnlwgt**         | Cecha        | Liczbowa     | *(brak)*             | *(brak)*                                                         | nie          |
| **education**      | Cecha        | Kategoryczna | Poziom Wykształcenia | Bachelors, Some-college, 11th, HS-grad, Prof-school, …           | nie          |
| **education-num**  | Cecha        | Liczbowa     | Poziom Wykształcenia | *(brak)*                                                         | nie          |
| **marital-status** | Cecha        | Kategoryczna | Inne                 | Married-civ-spouse, Divorced, Never-married, Separated, …        | nie          |
| **occupation**     | Cecha        | Kategoryczna | Inne                 | Tech-support, Craft-repair, Other-service, Sales, …              | tak          |
| **relationship**   | Cecha        | Kategoryczna | Inne                 | Wife, Own-child, Husband, Not-in-family, Other-relative, …       | nie          |
| **race**           | Cecha        | Kategoryczna | Rasa                 | White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black      | nie          |
| **sex**            | Cecha        | Binarna      | Płeć                 | Female, Male                                                     | nie          |
| **capital-gain**   | Cecha        | Liczbowa     | *(brak)*             | *(brak)*                                                         | nie          |
| **capital-loss**   | Cecha        | Liczbowa     | *(brak)*             | *(brak)*                                                         | nie          |
| **hours-per-week** | Cecha        | Liczbowa     | *(brak)*             | *(brak)*                                                         | nie          |
| **native-country** | Cecha        | Kategoryczna | Inne                 | United-States, Cambodia, England, Puerto-Rico, Canada, …         | tak          |
| **income**         | Zmienna celu | Binarna      | Zarobki              | >50K, <=50K                                                      | nie          |


In [32]:
print(adult.metadata['additional_info']['variable_info'])

Listing of attributes:

>50K, <=50K.

age: continuous.
workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
fnlwgt: continuous.
education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
education-num: continuous.
marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
sex: Female, Male.
capital-gain: continuous.
capital-loss: continuous.
hours-per-week: continuous.
native-country: Unite

In [12]:
print(adult.variables)

              name     role         type      demographic  \
0              age  Feature      Integer              Age   
1        workclass  Feature  Categorical           Income   
2           fnlwgt  Feature      Integer             None   
3        education  Feature  Categorical  Education Level   
4    education-num  Feature      Integer  Education Level   
5   marital-status  Feature  Categorical            Other   
6       occupation  Feature  Categorical            Other   
7     relationship  Feature  Categorical            Other   
8             race  Feature  Categorical             Race   
9              sex  Feature       Binary              Sex   
10    capital-gain  Feature      Integer             None   
11    capital-loss  Feature      Integer             None   
12  hours-per-week  Feature      Integer             None   
13  native-country  Feature  Categorical            Other   
14          income   Target       Binary           Income   

                       

In [6]:
X = adult.data.features 
y = adult.data.targets

df_adult = X
df_adult['income'] = y
df_adult.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### 3. Czyszczenie i wstępne przetwarzanie danych
Czyszczenie i wstępne przetwarzanie danych - imputacja braków danych, standaryzacja, kodowanie typu one-hot, transformacja wartości odstających, itp.

In [34]:
df_adult.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       47879 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      47876 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  48568 non-null  object
 14  income          48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


### 4. Eksploracyjna analiza danych
Graficzna i opisowa analiza eksploracyjna (EDA), m.in. graficzna prezentacja zależności pomiędzy wybraną zmienną celu i zmiennymi niezależnymi, wykonanie i opisanie wyników segmentacji (klastrowania) rekordów, itp.

### 5. Budowa modeli
Stworzenie min. 3 modeli i tuning hiperparametrów do zadania klasyfikacji lub regresji

### 6. Ocena i wybór najlepszego modelu
Graficzna i opisowa ocena oraz wybór modelu

### 7. Podsumowanie
Podsumowanie wyników, dyskusja na temat napotkanych problemów, wyzwań i zastosowanych rozwiązań